# Чтение данных из Tarantool

In [ ]:
import tarantool
import os
import pandas

In [ ]:
# подключение к БД Tarantool
HOST=os.environ['TARANTOOL_HOST']
PORT=os.environ['TARANTOOL_PORT']
LOGIN=os.environ['TARANTOOL_LOGIN']
PASSWORD=os.environ['TARANTOOL_PASSWORD']

connection = tarantool.connect(host = HOST, port=int(PORT), user=LOGIN, password=PASSWORD)

In [ ]:
# имя таблицы 
tableName = 'table1'

## Пример 1

Выбирать данные через `select *` не совсем правильно, т.к. нельзя быть уверенным, что последовательность полей будет такой как мы ожидаем, в итоге поля могут быть не правильно именованы, такое обычно приводит к тяжелым ошибкам.  

In [ ]:
sql = f"""
SELECT * from {tableName}
"""
print(sql)
rst = connection.execute(sql)
rst

In [ ]:
df = pandas.DataFrame( rst)
display(df)

# Пример 2

Для упрощения перевода списка в строку здесь добавляется виртуальная колонка c4.

In [ ]:
columns = ['c1', 'c2', 'c3']
sql = f"""
SELECT {' '.join([x+',' for x in columns])} null as c4 from {tableName}
"""
print(sql)
rst = connection.execute(sql)
rst

In [ ]:
df = pandas.DataFrame( rst, columns=columns+['c4'])
display(df)

In [ ]:
connection.close()

In [ ]:
# Пример 3

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, LongType, StringType, IntegerType, DateType, TimestampType, FloatType
from pyspark.sql.functions import col, cast, date_trunc, sum, dayofweek, hour, dayofmonth, lit

In [ ]:
# запуск spark
spark = SparkSession.builder.appName('fill').getOrCreate()

In [ ]:
# подключение к БД Tarantool
HOST=os.environ['TARANTOOL_HOST']
PORT=os.environ['TARANTOOL_PORT']
LOGIN=os.environ['TARANTOOL_LOGIN']
PASSWORD=os.environ['TARANTOOL_PASSWORD']

jdbcURLTarantool = f'jdbc:tarantool://{HOST}:{PORT}?user={LOGIN}&password={PASSWORD}'

In [ ]:
sql = f"""
select  *  from  {tableName} 
"""

dfData = spark.read.format("jdbc") \
                .option('url', jdbcURLTarantool) \
                .option('driver', 'org.tarantool.jdbc.SQLDriver') \
                .option('query', sql) \
                .load()

In [ ]:
dfData.show()

In [ ]:
display(dfData.toPandas())

In [ ]:
spark.stop()